In [209]:
import requests
import json
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

In [210]:
def bls_pull(registration_key, area_code, start_year, end_year):
    
    



    industries = {'11': 'natural_resources_mining',
                  '12': 'construction',
                  '13': 'manufacturing',
                  '21': 'trade_transportation_utilities',
                  '22': 'information',
                  '23': 'financial_activities',
                  '24': 'professional_business_services',
                  '25': 'education_health_services',
                  '26': 'leisure_hospitality',
                  '27': 'other_services',
                  '28': 'public_administration',
                  '29': 'unclassified'}

    data_names = {'1': 'all_employees',
                  '4': 'avg_weekly_pay'}




    json_dicts = {}

    for i_code, i_name in industries.items():

        for d_code, d_name in data_names.items():

            headers = {'Content-type': 'application/json'}
            data = json.dumps({"seriesid": ['ENU'+ area_code + d_code + '0510' + i_code],'registrationkey': registration_key, 
                               "startyear": start_year, "endyear": end_year})
            p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
            json_data = json.loads(p.text)

            json_dicts[i_name + '_' + d_name] = json_data['Results']['series'][0]['data']

    bad_keys = []
    for i in json_dicts:
        if len(json_dicts[i]) == 0:
            bad_keys.append(i) 

    for i in bad_keys:
        del json_dicts[i]




    df_dicts = {}

    for i in json_dicts:
        df_dicts[i] = pd.DataFrame(json_dicts[i])

    for i in df_dicts:

        if 'avg_weekly_pay' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        elif 'all_employees' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        cols = [j for j in df_dicts[i] if j != 'date']
        df_dicts[i][cols] = df_dicts[i][cols].apply(pd.to_numeric, errors='coerce')

    for df_key in df_dicts:

        wp_mask = df_key.split('all')[0] + 'avg_weekly_pay'

        if 'all_employees' in df_key:

            df_dicts[df_key][wp_mask] = 0

            for index, period in df_dicts[df_key]['date'].iteritems():
                if period[4:] in ['01', '02', '03']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'01')][wp_mask].iloc[0]
                if period[4:] in ['04', '05', '06']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'02')][wp_mask].iloc[0]
                if period[4:] in ['07', '08', '09']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'03')][wp_mask].iloc[0]
                if period[4:] in ['10', '11', '12']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'04')][wp_mask].iloc[0]

            df_dicts[df_key].rename(columns={wp_mask: df_key.split('all')[0] + 'avg_monthly_pay'}, inplace=True)
            df_dicts[df_key][df_key.split('all')[0] + 'aggr_value'] = df_dicts[df_key][df_key] * df_dicts[df_key][df_key.split('all')[0] + 'avg_monthly_pay']

    merged_df = pd.DataFrame(columns=['date'])

    for df_key in df_dicts:
        if 'all_employees' in df_key:
            merged_df = pd.merge(merged_df, df_dicts[df_key], how = 'outer', on ='date')

    temp_merge = merged_df.drop('date', axis=1)
    temp_cols = [i for i in temp_merge.columns if 'aggr_val' in i]
    merged_df['total_industries_aggr_value'] = temp_merge[temp_cols].apply(lambda x: sum(x), axis=1)
    
    #Reverses Dates:
    merged_df = merged_df[::-1]
    #Formates Dates year and month split
    merged_df['date'] = [x[:4]+'-'+x[4:8] for x in merged_df['date']]
    
    return pd.DataFrame(merged_df)


In [211]:
#EW API 9ff92c5c737a495297232e59bb594f55
#AIDIAN API 2738330500f74641a2f99b3ecd9831e2
#Linea API a5ee7376ece34349b16b469efbf2579d
#Linea API 213d94fbcbe941269843d7d2b26440b0

In [212]:
#Variables NEED INPUT
city = "Hudson"
start = "2011"
end = "2014"
code= "34017"
disaster = "2012-10"
# When did Disaster happen? 

In [213]:
#Pulls in data from BLS
df = bls_pull(registration_key='a5ee7376ece34349b16b469efbf2579d', start_year=start, end_year=end, area_code=code) 

KeyError: 'series'

In [ ]:
#Saves to pdf
df.to_csv("./BLSdata//" +str(city)+str(start)+'-'+str(end)+".csv")

In [ ]:
df.index = df['date']
df.drop(columns=['date'],inplace=True)

In [ ]:
#graphs stuff 

In [ ]:
traces = {}
tracecount = 0
for col in df.columns[:-1:3]:
    try:
        traces['trace'+ str(tracecount)] = dict(
                    x= df.index,
                    y= df[col],
                    mode='lines',
                    name= col)
        
        tracecount+=1
    except:
        print('error')

In [ ]:
traces['trace' +str(tracecount)] = dict(
x = [disaster]*2000,
y = np.linspace(0, 5000, num=20),
mode='markers',
name = "Disaster"
)

In [ ]:
data = [traces['trace'+str(x)] for x in range(0, (len(traces)))]

plot({'data':data,
       'layout' : {'title': str(city) +" employment figures for " +str(start) +"-" + str(end)},
     }, filename= "./BLSgraph//"+str(city)+"BLSplotly.html")

iplot({'data':data,
       'layout' : {'title': str(city) +" employment figures for " +str(start) +"-" + str(end)}})